In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

In [ ]:

    for snp in valid_snp_names:
        if snp in snp_ga:
            snp_feature = feature_column.categorical_column_with_vocabulary_list(snp, ['G/A', 'G/G', 'A/A' ])
        else:
            snp_feature = feature_column.categorical_column_with_vocabulary_list(snp, ['C/C', 'T/C', 'T/T' ])
        snp_feature_one_hot = feature_column.indicator_column(snp_feature)
        features.append(snp_feature_one_hot)

In [2]:
def get_allowed_snp_names():
    df = pd.read_csv('allowed_snp.csv')
    df['snp'] = df['snp'].apply(lambda x : x.split('_')[1])
    df = df.sort_values(by=['snp'])
    snps = df['snp'].tolist()
    for s in ['sex','smoke','drink','excercise','HP','age','CNE','BMI']:
        snps.append(s)
    return snps

snp_ga = ('rs10091503', 'rs1026435', 'rs10503675', 
              'rs11778205', 'rs1200135', 'rs1358714', 'rs1678674',
              'rs1738254', 'rs3737094', 'rs3807154', 'rs3823430', 
              'rs4452640', 'rs874808', 'rs9357283')


In [11]:
# 先构造一个正确的数据格式
result = []
def map_fn(row):
    snps = get_allowed_snp_names()
    tmp=[]
    for snp in snps:
        tmp.append(snp+'|'+str(row[snp]))
    result.append(tmp)
    # 返回一个list出来
def build_apriori_input_dataset():
    df = pd.read_csv('2.csv')
    df1 = pd.read_csv('1.csv')
    final_df = pd.concat([df, df1],ignore_index=True)
    final_case_df = final_df[(final_df['caco'] == 1) & (final_df['HL'] >= 40)]
    final_control_df = final_df[(final_df['caco'] == 0) & (final_df['HL']) < 40]
    final_df = pd.concat([final_case_df, final_control_df], ignore_index=True)
    final_df = final_df[final_df['age']>10]
    final_df['age'] = pd.cut(final_df['age'], [0, 20, 25, 30, 35, 40, 45, 50, 55,100], right=True, labels=[0,1,2,3,4,5,6,7,8])
    final_df['CNE'] = pd.cut(final_df['CNE'], [0,85, 90, 95, 100, 105, 110,10000], right=True, labels=[0,1,2,3,4,5,6])
    final_df['BMI'] = pd.cut(final_df['BMI'], [0,15, 20, 25, 30, 35, 40,10000], right=True, labels=[0,1,2,3,4,5,6])
    df = final_df[final_df['caco'] == 0]
    df.apply(map_fn,axis=1)
    return result

In [12]:
r = build_apriori_input_dataset()


In [ ]:
r

In [13]:
te = TransactionEncoder()
te_ary = te.fit(r).transform(r)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,BMI|1.0,BMI|2.0,BMI|3.0,BMI|4.0,BMI|5.0,BMI|6.0,BMI|nan,CNE|1,CNE|2,CNE|3,...,rs919390|C/C,rs919390|G/C,rs919390|G/G,rs9357283|A/A,rs9357283|G/A,rs9357283|G/G,sex|1,sex|2,smoke|0,smoke|1
0,False,False,True,False,False,False,False,False,False,True,...,False,True,False,False,True,False,True,False,True,False
1,False,True,False,False,False,False,False,False,False,True,...,True,False,False,False,False,True,True,False,False,True
2,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,False,False,True
3,False,False,True,False,False,False,False,False,False,False,...,False,True,False,True,False,False,True,False,False,True
4,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,False,False,True,False,False,False,False,False,False,False,...,True,False,False,False,False,True,True,False,False,True
726,False,True,False,False,False,False,False,False,False,True,...,True,False,False,False,False,True,True,False,True,False
727,False,False,True,False,False,False,False,False,False,False,...,False,True,False,True,False,False,True,False,True,False
728,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,True,True,False,False,True


In [14]:
dd = apriori(df, min_support=0.6, use_colnames=True)

In [15]:
dd.to_csv('control_apriori.csv',index=0, header=0)